In [1]:
from astropy.coordinates import SkyCoord
from astroquery.mast import Observations
import os
import gzip, shutil, glob

In [2]:
def Galex(pos, size = 0.1, unzip = True):
    """
    This function simplifies the fits file download process from GALEX.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    size : float, optional
        The searching cone size, by default 0.1 [deg]
    unzip : bool, optional
        Whether to unzip the downloaded file, by default True.
    """

    # Full list
    obs_table = Observations.query_region(pos,radius= size)
    data_products_by_obs = Observations.get_product_list(obs_table[:])
    # query list
    data_products = data_products_by_obs[(data_products_by_obs['obs_collection'] == 'GALEX')]

    # Preventing from repeating download
    # Here we only download three datatypes: intensity map, sky background map and background subtracted image
    a = data_products.to_pandas()
    b = a[a.productFilename.str.contains('int.fits|skybg.fits|intbgsub.fits')]

    # Start downloading
    for _, row in b.iterrows():
        Observations.download_file(row.dataURI)

    if unzip:
        for file in glob.glob("*.gz"):
            with gzip.open(file, 'r') as f_in, open(file.replace(".gz", ""), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)  # unzipped gz files
            os.remove(file)   # delete gz files  
    else:
        pass

In [3]:
import pandas as pd
import numpy as np

galaxy = pd.read_csv('GLSBs_with_names.csv')
# Remove ALL spaces inside the names
galaxies = galaxy['Object_Name'].str.replace(" ", "", regex=False).values
galaxies


['NGC6902']

In [ ]:
import os, time
from astropy.coordinates import SkyCoord
from astroquery.exceptions import TimeoutError

parent_dir = os.path.abspath("downloaded_galex")

for galaxy in galaxies:
    galaxy_clean = galaxy#.strip().replace(" ", "_")
    galaxy_dir = os.path.join(parent_dir, galaxy_clean)
    os.makedirs(galaxy_dir, exist_ok=True)
    os.chdir(galaxy_dir)
    print(f"\nDownloading GALEX data for {galaxy_clean} in {os.getcwd()}")
    
    for attempt in range(3):  # up to 3 tries
        try:
            Galex(pos=SkyCoord.from_name(galaxy))
            print(f"✅ Completed: {galaxy_clean}")
            break
        except TimeoutError:
            print(f"⏳ Timeout on {galaxy_clean} (attempt {attempt+1}/3), retrying...")
            time.sleep(15)
        except Exception as e:
            print(f"❌ Failed for {galaxy_clean}: {e}")
            break
    
    os.chdir(parent_dir)
